### <font color='blue'>Import all packages</font> ###

In [1]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 16.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
     |████████████████████████████████| 596 kB 74.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     \ 665.5 MB 393 kB/s
     |████████████████████████████████| 1.3 MB 13.2 MB/s 
     |████████████████████████████████| 463 kB 90.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling ten

In [3]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [5]:
!cp "/content/drive/My Drive/summarized_sc.txt" "./summarized_sc.txt"
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"
!cp "/content/drive/My Drive/sc_model_0.txt" "./sc_model_0.txt"
!cp "/content/drive/My Drive/sc_model_1.txt" "./sc_model_1.txt"
!cp "/content/drive/My Drive/sc_model_2.txt" "./sc_model_2.txt"
!cp "/content/drive/My Drive/sc_model_3.txt" "./sc_model_3.txt"
!cp "/content/drive/My Drive/sc_model_4.txt" "./sc_model_4.txt"
!cp "/content/drive/My Drive/sc_model_5.txt" "./sc_model_5.txt"

In [6]:
fh = open('sc_model_0.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)


8419
8419
Average Length 489.8335906877301
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '1

In [7]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(279, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model 

In [8]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_0=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_0=[]
new_val_inp_0=[]
new_train_label_0=[]
new_val_label_0=[]
new_train_mask_0=[]
new_train_fnum_0=[]
new_val_fnum_0=[]
new_val_mask_0=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_0.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_0.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_0.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_0.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_0.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_0.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_0.append(val_mask[i])
    new_val_fnum_0.append(val_fnum[i])

new_train_inp_0=np.array(new_train_inp_0)
new_val_inp_0=np.array(new_val_inp_0)
new_train_label_0=np.array(new_train_label_0)
new_val_label_0=np.array(new_val_label_0)
new_train_mask_0=np.array(new_train_mask_0)
new_train_fnum_0=np.array(new_train_fnum_0)
new_val_fnum_0=np.array(new_val_fnum_0)
new_val_mask_0=np.array(new_val_mask_0)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)

  model_0=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_0=[]
  new_val_inp_0=[]
  new_train_label_0=[]
  new_val_label_0=[]
  new_train_mask_0=[]
  new_train_fnum_0=[]
  new_val_fnum_0=[]
  new_val_mask_0=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_0.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_0.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_0.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_0.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_0.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_0.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_0.append(val_mask[i])
      new_val_fnum_0.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_0=np.array(new_train_inp_0)
  new_val_inp_0=np.array(new_val_inp_0)
  new_train_label_0=np.array(new_train_label_0)
  new_val_label_0=np.array(new_val_label_0)
  new_train_mask_0=np.array(new_train_mask_0)
  new_train_fnum_0=np.array(new_train_fnum_0)
  new_val_fnum_0=np.array(new_val_fnum_0)
  new_val_mask_0=np.array(new_val_mask_0)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_0-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_0.fit([new_train_inp_0,new_train_mask_0],new_train_label_0,batch_size=8,epochs=5,validation_data=([new_val_inp_0,new_val_mask_0],new_val_label_0),callbacks=callbacks)

  pred_labels=[]

  model_saved_0= create_model()
  model_saved_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_0.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_0-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_0.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]
Mon Jun  6 23:18:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    37W / 300W |   2675MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                      

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]
Tue Jun  7 01:19:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    37W / 300W |  15237MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                      

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


948/948 [==============================] - 340s 341ms/step - loss: 12.6865 - accuracy: 0.2014 - val_loss: 10.6493 - val_accuracy: 0.3622
Epoch 2/5
948/948 [==============================] - 323s 341ms/step - loss: 9.7811 - accuracy: 0.4225 - val_loss: 9.0557 - val_accuracy: 0.4537
Epoch 3/5
948/948 [==============================] - 323s 341ms/step - loss: 8.3037 - accuracy: 0.5218 - val_loss: 8.0662 - val_accuracy: 0.4810
Epoch 4/5
948/948 [==============================] - 322s 340ms/step - loss: 7.1940 - accuracy: 0.6042 - val_loss: 7.3121 - val_accuracy: 0.5048
Epoch 5/5
948/948 [==============================] - 323s 340ms/step - loss: 6.2317 - accuracy: 0.6834 - val_loss: 6.6557 - val_accuracy: 0.5083
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           [] 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
fh = open('sc_model_1.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)
print(summarized_data['label'])


7332
7332
Average Length 510.89102564102564
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '

In [10]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_1=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

#train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_1=[]
new_val_inp_1=[]
new_train_label_1=[]
new_val_label_1=[]
new_train_mask_1=[]
new_train_fnum_1=[]
new_val_fnum_1=[]
new_val_mask_1=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_1.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_1.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_1.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_1.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_1.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_1.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_1.append(val_mask[i])
    new_val_fnum_1.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_1=np.array(new_train_inp_1)
new_val_inp_1=np.array(new_val_inp_1)
new_train_label_1=np.array(new_train_label_1)
new_val_label_1=np.array(new_val_label_1)
new_train_mask_1=np.array(new_train_mask_1)
new_train_fnum_1=np.array(new_train_fnum_1)
new_val_fnum_1=np.array(new_val_fnum_1)
new_val_mask_1=np.array(new_val_mask_1)

print(new_val_fnum_1)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 2883 4515
 3001 3410 6991 2875 1125 3885 6647 4857 4504  214 3230 8340 2969 3847
 4609 5092 1346 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445
 6483  542 1663 4952 5072 1967 4071 7590  860 6418 5030 7297  626 5463
 6029 6302 3819 7256 2950 5667 1235 3993 1439 1323 4263 3835 2134 2624
 5801 4293 8129  320 5866 3247 5777   19 6059 6520 4837 3843 5695 1666
  858 1418 5409 5274 6205 3167 2356 8418 5997 6685  223 3444 3310 7721
 5928 2360 7197 4252 8343  909 4093 4354 4752 1421  649  829 1916 1003
 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293 6380 4602 6057
 1851 4577 4236 7978 1350 5083 6552 1058  361 7706 4186 2590 4452 4396
  749  870 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 1845 5119 2516 4673 7947 8215 1882 3350 7617 7152 5199 4282 1092 3342
 8003 5025 5051  535  933 3969 1927 3615 1221 8062 1489 3296 7442 5459
 2031 7351 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)
  
  model_1=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  #train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_1=[]
  new_val_inp_1=[]
  new_train_label_1=[]
  new_val_label_1=[]
  new_train_mask_1=[]
  new_train_fnum_1=[]
  new_val_fnum_1=[]
  new_val_mask_1=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_1.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_1.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_1.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_1.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_1.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_1.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_1.append(val_mask[i])
      new_val_fnum_1.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_1=np.array(new_train_inp_1)
  new_val_inp_1=np.array(new_val_inp_1)
  new_train_label_1=np.array(new_train_label_1)
  new_val_label_1=np.array(new_val_label_1)
  new_train_mask_1=np.array(new_train_mask_1)
  new_train_fnum_1=np.array(new_train_fnum_1)
  new_val_fnum_1=np.array(new_val_fnum_1)
  new_val_mask_1=np.array(new_val_mask_1)

  print(new_val_fnum_1)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_1-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_1.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_1.fit([new_train_inp_1,new_train_mask_1],new_train_label_1,batch_size=8,epochs=5,validation_data=([new_val_inp_1,new_val_mask_1],new_val_label_1),callbacks=callbacks)

  pred_labels=[]

  model_saved_1= create_model()
  model_saved_1.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_1.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_1-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_1.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 2883 4515
 3001 3410 6991 2875 1125 3885 6647 4857 4504  214 3230 8340 2969 3847
 4609 5092 1346 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445
 6483  542 1663 4952 5072 1967 4071 7590  860 6418 5030 7297  626 5463
 6029 6302 3819 7256 2950 5667 1235 3993 1439 1323 4263 3835 2134 2624
 5801 4293 8129  320 5866 3247 5777   19 6059 6520 4837 3843 5695 1666
  858 1418 5409 5274 6205 3167 2356 8418 5997 6685  223 3444 3310 7721
 5928 2360 7197 4252 8343  909 4093 4354 4752 1421  649  829 1916 1003
 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293 6380 4602 6057
 1851 4577 4236 7978 1350 5083 6552 1058  361 7706 4186 2590 4452 4396
  749  870 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 1845 5119 2516 4673 7947 8215 1882 3350 7617 7152 5199 4282 1092 3342
 8003 5025 5051  535  933 3969 1927 3615 1221 8062 1489 3296 7442 5459
 2031 7351 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ...   4  74 263]
[   1    2    3 ... 8417 8418 8419]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 2883 4515
 3001 3410 6991 2875 1125 3885 6647 4857 4504  214 3230 8340 2969 3847
 4609 5092 1346 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445
 6483  542 1663 4952 5072 1967 4071 7590  860 6418 5030 7297  626 5463
 6029 6302 3819 7256 2950 5667 1235 3993 1439 1323 4263 3835 2134 2624
 5801 4293 8129  320 5866 3247 5777   19 6059 6520 4837 3843 5695 1666
  858 1418 5409 5274 6205 3167 2356 8418 5997 6685  223 3444 3310 7721
 5928 2360 7197 4252 8343  909 4093 4354 4752 1421  649  829 1916 1003
 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293 6380 4602 6057
 1851 4577 4236 7978 1350 5083 6552 1058  361 7706 4186 2590 4452 4396
  749  870 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 1845 5119 2516 4673 7947 8215 1882 3350 7617 7152 5199 4282 1092 3342
 8003 5025 5051  535  933 3969 1927 3615 1221 8062 1489 3296 7442 5459
 2031 7351 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


826/826 [==============================] - 299s 343ms/step - loss: 12.8328 - accuracy: 0.1806 - val_loss: 10.7679 - val_accuracy: 0.3817
Epoch 2/5
826/826 [==============================] - 282s 342ms/step - loss: 9.8841 - accuracy: 0.4118 - val_loss: 9.1699 - val_accuracy: 0.4501
Epoch 3/5
826/826 [==============================] - 282s 342ms/step - loss: 8.4071 - accuracy: 0.5292 - val_loss: 8.2738 - val_accuracy: 0.4761
Epoch 4/5
826/826 [==============================] - 282s 341ms/step - loss: 7.3236 - accuracy: 0.6022 - val_loss: 7.4883 - val_accuracy: 0.5062
Epoch 5/5
826/826 [==============================] - 282s 342ms/step - loss: 6.4327 - accuracy: 0.6676 - val_loss: 6.9186 - val_accuracy: 0.5157
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           [] 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
fh = open('sc_model_2.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)


6722
6722
Average Length 511.5861350788456
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '1

In [12]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_2=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

#train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_2=[]
new_val_inp_2=[]
new_train_label_2=[]
new_val_label_2=[]
new_train_mask_2=[]
new_train_fnum_2=[]
new_val_fnum_2=[]
new_val_mask_2=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_2.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_2.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_2.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_2.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_2.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_2.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_2.append(val_mask[i])
    new_val_fnum_2.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_2=np.array(new_train_inp_2)
new_val_inp_2=np.array(new_val_inp_2)
new_train_label_2=np.array(new_train_label_2)
new_val_label_2=np.array(new_val_label_2)
new_train_mask_2=np.array(new_train_mask_2)
new_train_fnum_2=np.array(new_train_fnum_2)
new_val_fnum_2=np.array(new_val_fnum_2)
new_val_mask_2=np.array(new_val_mask_2)

print(new_val_fnum_2)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 4515 3001
 6991 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 5092 1346
 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445 6483 1663 4952
 5072 1967 4071 7590  860 6418 5030 7297  626 5463 6029 6302 3819 7256
 5667 1235 3993 1439 1323 4263 3835 2624 5801 4293 8129  320 5866 3247
 5777   19 6059 4837 3843 1666  858 1418 5409 5274 6205 3167 2356 5997
 6685  223 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752 1421
  649  829 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293
 6380 4602 6057 1851 4577 4236 7978 5083 6552 1058  361 7706 4186 4452
 4396  749 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 5119 4673 7947 1882 3350 7617 7152 5199 4282 1092 8003 5025 5051  933
 3969 1927 3615 1221 8062 1489 3296 7442 5459 2031 7351  234 2313 8015
 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425 1321 6580
  928 1170 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)
  
  model_2=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  #train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_2=[]
  new_val_inp_2=[]
  new_train_label_2=[]
  new_val_label_2=[]
  new_train_mask_2=[]
  new_train_fnum_2=[]
  new_val_fnum_2=[]
  new_val_mask_2=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_2.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_2.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_2.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_2.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_2.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_2.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_2.append(val_mask[i])
      new_val_fnum_2.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_2=np.array(new_train_inp_2)
  new_val_inp_2=np.array(new_val_inp_2)
  new_train_label_2=np.array(new_train_label_2)
  new_val_label_2=np.array(new_val_label_2)
  new_train_mask_2=np.array(new_train_mask_2)
  new_train_fnum_2=np.array(new_train_fnum_2)
  new_val_fnum_2=np.array(new_val_fnum_2)
  new_val_mask_2=np.array(new_val_mask_2)
  
  print(new_val_fnum_2)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_2-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_2.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_2.fit([new_train_inp_2,new_train_mask_2],new_train_label_2,batch_size=8,epochs=5,validation_data=([new_val_inp_2,new_val_mask_2],new_val_label_2),callbacks=callbacks)

  pred_labels=[]

  model_saved_2= create_model()
  model_saved_2.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_2.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_2-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_2.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 4515 3001
 6991 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 5092 1346
 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445 6483 1663 4952
 5072 1967 4071 7590  860 6418 5030 7297  626 5463 6029 6302 3819 7256
 5667 1235 3993 1439 1323 4263 3835 2624 5801 4293 8129  320 5866 3247
 5777   19 6059 4837 3843 1666  858 1418 5409 5274 6205 3167 2356 5997
 6685  223 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752 1421
  649  829 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293
 6380 4602 6057 1851 4577 4236 7978 5083 6552 1058  361 7706 4186 4452
 4396  749 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 5119 4673 7947 1882 3350 7617 7152 5199 4282 1092 8003 5025 5051  933
 3969 1927 3615 1221 8062 1489 3296 7442 5459 2031 7351  234 2313 8015
 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425 1321 6580
  928 1170 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344  454 5464  317 4515 3001
 6991 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 5092 1346
 5993 7358 2752 7651 7301 8188 7073 4782 6069 2346 6445 6483 1663 4952
 5072 1967 4071 7590  860 6418 5030 7297  626 5463 6029 6302 3819 7256
 5667 1235 3993 1439 1323 4263 3835 2624 5801 4293 8129  320 5866 3247
 5777   19 6059 4837 3843 1666  858 1418 5409 5274 6205 3167 2356 5997
 6685  223 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752 1421
  649  829 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 1010 6293
 6380 4602 6057 1851 4577 4236 7978 5083 6552 1058  361 7706 4186 4452
 4396  749 8351 3834  812 5108  463 1743 6600 8264 8064 7121 8279 5556
 5119 4673 7947 1882 3350 7617 7152 5199 4282 1092 8003 5025 5051  933
 3969 1927 3615 1221 8062 1489 3296 7442 5459 2031 7351  234 2313 8015
 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425 1321 6580
  928 1170 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


757/757 [==============================] - 275s 342ms/step - loss: 13.0396 - accuracy: 0.1497 - val_loss: 10.8996 - val_accuracy: 0.3488
Epoch 2/5
757/757 [==============================] - 258s 341ms/step - loss: 10.0088 - accuracy: 0.3831 - val_loss: 9.2306 - val_accuracy: 0.4581
Epoch 3/5
757/757 [==============================] - 259s 342ms/step - loss: 8.4614 - accuracy: 0.5139 - val_loss: 8.2643 - val_accuracy: 0.4820
Epoch 4/5
757/757 [==============================] - 259s 342ms/step - loss: 7.3885 - accuracy: 0.5965 - val_loss: 7.4757 - val_accuracy: 0.5015
Epoch 5/5
757/757 [==============================] - 259s 342ms/step - loss: 6.4712 - accuracy: 0.6713 - val_loss: 6.8928 - val_accuracy: 0.5210
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           []

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
fh = open('sc_model_3.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

6005
6005
Average Length 511.7708576186511
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '1

In [14]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_3=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

#train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_3=[]
new_val_inp_3=[]
new_train_label_3=[]
new_val_label_3=[]
new_train_mask_3=[]
new_train_fnum_3=[]
new_val_fnum_3=[]
new_val_mask_3=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_3.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_3.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_3.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_3.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_3.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_3.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_3.append(val_mask[i])
    new_val_fnum_3.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_3=np.array(new_train_inp_3)
new_val_inp_3=np.array(new_val_inp_3)
new_train_label_3=np.array(new_train_label_3)
new_val_label_3=np.array(new_val_label_3)
new_train_mask_3=np.array(new_train_mask_3)
new_train_fnum_3=np.array(new_train_fnum_3)
new_val_fnum_3=np.array(new_val_fnum_3)
new_val_mask_3=np.array(new_val_mask_3)

print(new_val_fnum_3)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344 5464  317 4515 3001 6991
 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358
 2752 7651 8188 7073 4782 6069 2346 6483 1663 4952 5072 1967 4071 7590
  860 6418 5030 7297  626 5463 6029 6302 7256 5667 1235 3993 1323 4263
 3835 2624 5801 4293 8129 5866 5777 6059 4837 3843 1418 5409 5274 6205
 2356 5997 6685 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752
  649 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 1851 4577 4236 7978 5083 6552  361 7706 4186 4452 4396 8351 3834
  812 5108  463 6600 8264 8064 7121 8279 5556 5119 7947 3350 7152 5199
 4282 1092 5025 5051  933 1927 3615 1221 8062 1489 7442 5459 2031 7351
  234 8015 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425
 1321 6580  928 1170 5404 8273 6742   94    9 7861 7975 1018 2140 1415
 7061 7767 4990 8124 5927 6051 3269 6916 5959 2846 6818 7052 7775 7582
 8116 4240 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)
  
  model_3=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  #train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_3=[]
  new_val_inp_3=[]
  new_train_label_3=[]
  new_val_label_3=[]
  new_train_mask_3=[]
  new_train_fnum_3=[]
  new_val_fnum_3=[]
  new_val_mask_3=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_3.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_3.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_3.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_3.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_3.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_3.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_3.append(val_mask[i])
      new_val_fnum_3.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_3=np.array(new_train_inp_3)
  new_val_inp_3=np.array(new_val_inp_3)
  new_train_label_3=np.array(new_train_label_3)
  new_val_label_3=np.array(new_val_label_3)
  new_train_mask_3=np.array(new_train_mask_3)
  new_train_fnum_3=np.array(new_train_fnum_3)
  new_val_fnum_3=np.array(new_val_fnum_3)
  new_val_mask_3=np.array(new_val_mask_3)
  
  print(new_val_fnum_3)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_3-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_3.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_3.fit([new_train_inp_3,new_train_mask_3],new_train_label_3,batch_size=8,epochs=5,validation_data=([new_val_inp_3,new_val_mask_3],new_val_label_3),callbacks=callbacks)

  pred_labels=[]

  model_saved_3= create_model()
  model_saved_3.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_3.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_3-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_3.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344 5464  317 4515 3001 6991
 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358
 2752 7651 8188 7073 4782 6069 2346 6483 1663 4952 5072 1967 4071 7590
  860 6418 5030 7297  626 5463 6029 6302 7256 5667 1235 3993 1323 4263
 3835 2624 5801 4293 8129 5866 5777 6059 4837 3843 1418 5409 5274 6205
 2356 5997 6685 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752
  649 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 1851 4577 4236 7978 5083 6552  361 7706 4186 4452 4396 8351 3834
  812 5108  463 6600 8264 8064 7121 8279 5556 5119 7947 3350 7152 5199
 4282 1092 5025 5051  933 1927 3615 1221 8062 1489 7442 5459 2031 7351
  234 8015 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425
 1321 6580  928 1170 5404 8273 6742   94    9 7861 7975 1018 2140 1415
 7061 7767 4990 8124 5927 6051 3269 6916 5959 2846 6818 7052 7775 7582
 8116 4240 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 216 ... 101  26   4]
[   1    2    3 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 3942 8344 5464  317 4515 3001 6991
 2875 1125 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358
 2752 7651 8188 7073 4782 6069 2346 6483 1663 4952 5072 1967 4071 7590
  860 6418 5030 7297  626 5463 6029 6302 7256 5667 1235 3993 1323 4263
 3835 2624 5801 4293 8129 5866 5777 6059 4837 3843 1418 5409 5274 6205
 2356 5997 6685 3444 3310 7721 5928 2360 7197 4252 8343  909 4354 4752
  649 1916 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 1851 4577 4236 7978 5083 6552  361 7706 4186 4452 4396 8351 3834
  812 5108  463 6600 8264 8064 7121 8279 5556 5119 7947 3350 7152 5199
 4282 1092 5025 5051  933 1927 3615 1221 8062 1489 7442 5459 2031 7351
  234 8015 7416 7921 6574 6257 8396  941 4520 3300 5640  240 7810 7425
 1321 6580  928 1170 5404 8273 6742   94    9 7861 7975 1018 2140 1415
 7061 7767 4990 8124 5927 6051 3269 6916 5959 2846 6818 7052 7775 7582
 8116 4240 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


677/677 [==============================] - 248s 343ms/step - loss: 13.0920 - accuracy: 0.1494 - val_loss: 11.0912 - val_accuracy: 0.3479
Epoch 2/5
677/677 [==============================] - 232s 343ms/step - loss: 10.1779 - accuracy: 0.4043 - val_loss: 9.4916 - val_accuracy: 0.4538
Epoch 3/5
677/677 [==============================] - 232s 342ms/step - loss: 8.7126 - accuracy: 0.5222 - val_loss: 8.5093 - val_accuracy: 0.5092
Epoch 4/5
677/677 [==============================] - 232s 342ms/step - loss: 7.6869 - accuracy: 0.5928 - val_loss: 7.8210 - val_accuracy: 0.5227
Epoch 5/5
677/677 [==============================] - 228s 337ms/step - loss: 6.8250 - accuracy: 0.6623 - val_loss: 7.3231 - val_accuracy: 0.4941
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           []

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
fh = open('sc_model_4.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

5275
5275
Average Length 511.7759241706161
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '1

In [16]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_4=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

#train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_4=[]
new_val_inp_4=[]
new_train_label_4=[]
new_val_label_4=[]
new_train_mask_4=[]
new_train_fnum_4=[]
new_val_fnum_4=[]
new_val_mask_4=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_4.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_4.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_4.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_4.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_4.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_4.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_4.append(val_mask[i])
    new_val_fnum_4.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_4=np.array(new_train_inp_4)
new_val_inp_4=np.array(new_val_inp_4)
new_train_label_4=np.array(new_train_label_4)
new_val_label_4=np.array(new_val_label_4)
new_train_mask_4=np.array(new_train_mask_4)
new_train_fnum_4=np.array(new_train_fnum_4)
new_val_fnum_4=np.array(new_val_fnum_4)
new_val_mask_4=np.array(new_val_mask_4)

print(new_val_fnum_4)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ... 101  26   4]
[   1    2    4 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 8344 5464  317 4515 3001 6991 2875
 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358 2752 7651
 8188 7073 6069 2346 6483 4952 5072 1967 4071 7590 5030 7297  626 5463
 6029 6302 7256 3993 4263 3835 5801 4293 8129 5866 5777 4837 3843 1418
 5274 6205 2356 5997 6685 3310 7721 5928 2360 7197 4252 8343  909 4354
 4752  649 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 4577 4236 7978 5083 6552  361 7706 4186 8351  812 5108 6600 8264
 8064 7121 8279 5556 5119 7947 3350 7152 5199 4282 5025 5051 1927 3615
 1221 8062 1489 2031 7351  234 8015 7416 7921 6574 6257 8396  941 4520
 3300 5640  240 7810 7425 1321 6580 5404 8273 6742   94    9 7861 7975
 1018 2140 1415 7061 7767 4990 8124 5927 6051 3269 6916 2846 6818 7052
 7775 7582 8116 4767 5248 1812 6055 8331  998 1564 7326  292 3065 6764
 6854 4915 3583 5902 4229 2144 7565 3712 7224 4502 1406 1651 3827 6525
 6318 2686 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)
  
  model_4=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  #train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_4=[]
  new_val_inp_4=[]
  new_train_label_4=[]
  new_val_label_4=[]
  new_train_mask_4=[]
  new_train_fnum_4=[]
  new_val_fnum_4=[]
  new_val_mask_4=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_4.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_4.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_4.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_4.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_4.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_4.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_4.append(val_mask[i])
      new_val_fnum_4.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_4=np.array(new_train_inp_4)
  new_val_inp_4=np.array(new_val_inp_4)
  new_train_label_4=np.array(new_train_label_4)
  new_val_label_4=np.array(new_val_label_4)
  new_train_mask_4=np.array(new_train_mask_4)
  new_train_fnum_4=np.array(new_train_fnum_4)
  new_val_fnum_4=np.array(new_val_fnum_4)
  new_val_mask_4=np.array(new_val_mask_4)

  print(new_val_fnum_4)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_4-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_4.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_4.fit([new_train_inp_4,new_train_mask_4],new_train_label_4,batch_size=8,epochs=5,validation_data=([new_val_inp_4,new_val_mask_4],new_val_label_4),callbacks=callbacks)

  pred_labels=[]

  model_saved_4= create_model()
  model_saved_4.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_4.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_4-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_4.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ... 101  26   4]
[   1    2    4 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 8344 5464  317 4515 3001 6991 2875
 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358 2752 7651
 8188 7073 6069 2346 6483 4952 5072 1967 4071 7590 5030 7297  626 5463
 6029 6302 7256 3993 4263 3835 5801 4293 8129 5866 5777 4837 3843 1418
 5274 6205 2356 5997 6685 3310 7721 5928 2360 7197 4252 8343  909 4354
 4752  649 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 4577 4236 7978 5083 6552  361 7706 4186 8351  812 5108 6600 8264
 8064 7121 8279 5556 5119 7947 3350 7152 5199 4282 5025 5051 1927 3615
 1221 8062 1489 2031 7351  234 8015 7416 7921 6574 6257 8396  941 4520
 3300 5640  240 7810 7425 1321 6580 5404 8273 6742   94    9 7861 7975
 1018 2140 1415 7061 7767 4990 8124 5927 6051 3269 6916 2846 6818 7052
 7775 7582 8116 4767 5248 1812 6055 8331  998 1564 7326  292 3065 6764
 6854 4915 3583 5902 4229 2144 7565 3712 7224 4502 1406 1651 3827 6525
 6318 2686 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ... 101  26   4]
[   1    2    4 ... 8413 8414 8417]
[6622   35 6713 4989 8233 7444 5061 8344 5464  317 4515 3001 6991 2875
 3885 6647 4857 4504  214 8340 2969 3847 4609 1346 5993 7358 2752 7651
 8188 7073 6069 2346 6483 4952 5072 1967 4071 7590 5030 7297  626 5463
 6029 6302 7256 3993 4263 3835 5801 4293 8129 5866 5777 4837 3843 1418
 5274 6205 2356 5997 6685 3310 7721 5928 2360 7197 4252 8343  909 4354
 4752  649 3488 2496 5765 6612 4004 4052 1403 6697 5268 6293 6380 4602
 6057 4577 4236 7978 5083 6552  361 7706 4186 8351  812 5108 6600 8264
 8064 7121 8279 5556 5119 7947 3350 7152 5199 4282 5025 5051 1927 3615
 1221 8062 1489 2031 7351  234 8015 7416 7921 6574 6257 8396  941 4520
 3300 5640  240 7810 7425 1321 6580 5404 8273 6742   94    9 7861 7975
 1018 2140 1415 7061 7767 4990 8124 5927 6051 3269 6916 2846 6818 7052
 7775 7582 8116 4767 5248 1812 6055 8331  998 1564 7326  292 3065 6764
 6854 4915 3583 5902 4229 2144 7565 3712 7224 4502 1406 1651 3827 6525
 6318 2686 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


595/595 [==============================] - 222s 346ms/step - loss: 13.2972 - accuracy: 0.1688 - val_loss: 11.4806 - val_accuracy: 0.3385
Epoch 2/5
595/595 [==============================] - 205s 345ms/step - loss: 10.5686 - accuracy: 0.4113 - val_loss: 9.9150 - val_accuracy: 0.4410
Epoch 3/5
595/595 [==============================] - 205s 344ms/step - loss: 9.1196 - accuracy: 0.5298 - val_loss: 9.0273 - val_accuracy: 0.4797
Epoch 4/5
595/595 [==============================] - 205s 344ms/step - loss: 8.0676 - accuracy: 0.6143 - val_loss: 8.3729 - val_accuracy: 0.4952
Epoch 5/5
595/595 [==============================] - 205s 344ms/step - loss: 7.2255 - accuracy: 0.6751 - val_loss: 7.8623 - val_accuracy: 0.5010
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           []

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
fh = open('sc_model_5.txt')
filenums=[]
records=[]
for record in fh:
    record=record.split("---")
    filenum=record[0]
    modelnum=record[1]
    text=record[2]
    filenums.append(filenum)
    records.append(text)
    
print(len(filenums))
print(len(records))

summarized_data = pd.DataFrame(list(zip(filenums, records)),columns =['filenum','text'])
len_list = [len(ele.split()) for ele in records]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res)

#temp_file = open("labels_sc.txt", "r")
temp_file = open("labels_sc_279.txt", "r")

data = temp_file.read()

label_list = data.split("\n")
print(label_list)
label_list = label_list[0:-1]
print(label_list)
label_list = [int(i) for i in label_list]
temp_file.close()


new_label_list=[]
for num in filenums:
  new_label_list.append(label_list[int(num)-1])
summarized_data['label'] = new_label_list

print(summarized_data)

4504
4504
Average Length 511.5863676731794
['209', '63', '216', '108', '196', '200', '203', '246', '188', '193', '25', '106', '56', '53', '223', '161', '200', '208', '78', '147', '147', '179', '147', '80', '56', '209', '209', '200', '214', '202', '203', '194', '26', '237', '165', '258', '238', '238', '76', '193', '196', '213', '161', '213', '209', '194', '196', '25', '151', '260', '135', '173', '30', '108', '272', '259', '176', '213', '213', '21', '5', '171', '200', '195', '240', '194', '176', '240', '240', '80', '173', '98', '214', '21', '76', '76', '73', '173', '173', '78', '61', '258', '191', '3', '78', '173', '193', '118', '200', '264', '260', '49', '211', '258', '18', '173', '78', '3', '3', '1', '213', '135', '182', '239', '208', '278', '50', '194', '200', '124', '189', '117', '264', '244', '76', '193', '221', '78', '78', '173', '197', '78', '161', '231', '135', '22', '78', '25', '25', '197', '200', '147', '203', '191', '196', '193', '263', '32', '197', '191', '4', '56', '196', '1

In [18]:
gc.collect()
tf.keras.backend.clear_session()
dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
max_len=512
sentences=summarized_data['text']
labels=summarized_data['label']
filenumbers=summarized_data['filenum']
len(sentences),len(labels),len(filenumbers)

model_5=create_model()
input_ids=[]
attention_masks=[]

for sent in sentences:
  dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
  input_ids.append(dbert_inps['input_ids'])
  attention_masks.append(dbert_inps['attention_mask'])
input_ids=np.asarray(input_ids)

attention_masks=np.array(attention_masks)
labels=np.array(labels)
filenumbers=[int(i) for i in filenumbers]
filenumbers=np.array(filenumbers)
print(labels)
print(filenumbers)

train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

new_train_inp_5=[]
new_val_inp_5=[]
new_train_label_5=[]
new_val_label_5=[]
new_train_mask_5=[]
new_train_fnum_5=[]
new_val_fnum_5=[]
new_val_mask_5=[]
for i in range(len(train_fnum)):
  if train_fnum[i] in filenumbers:
    new_train_inp_5.append(train_inp[i])
    #new_val_inp.append(val_inp[i])
    new_train_label_5.append(train_label[i])
    #new_val_label.append(val_label[i])
    new_train_mask_5.append(train_mask[i])
    #new_val_mask.append(val_mask[i])
    new_train_fnum_5.append(train_fnum[i])

for i in range(len(val_fnum)):
  if val_fnum[i] in filenumbers:
    #new_train_inp.append(train_inp[i])
    new_val_inp_5.append(val_inp[i])
    #new_train_label.append(train_label[i])
    new_val_label_5.append(val_label[i])
    #new_train_mask.append(train_mask[i])
    new_val_mask_5.append(val_mask[i])
    new_val_fnum_5.append(val_fnum[i])

# print(new_train_label)
# print(len(new_train_label))
# print(new_val_label)
# print(len(new_val_label))
new_train_inp_5=np.array(new_train_inp_5)
new_val_inp_5=np.array(new_val_inp_5)
new_train_label_5=np.array(new_train_label_5)
new_val_label_5=np.array(new_val_label_5)
new_train_mask_5=np.array(new_train_mask_5)
new_train_fnum_5=np.array(new_train_fnum_5)
new_val_fnum_5=np.array(new_val_fnum_5)
new_val_mask_5=np.array(new_val_mask_5)

print(new_val_fnum_5)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ...  26   4  57]
[   1    2    4 ... 8414 8417 8406]
[5596 6574 6333 3587 5524 7527 3634 3578 8103 3832 5607 7486 5997 7826
 4157 5688 5253 7691 1418 5606 4518 7620 5550 7405 4020 7918 4878 7248
 5565 8257 7965 6207 5878 2772 6330 3909 7196 6719 8393 8172 1609 7985
 6052 2269 4125 5143 6021 6606 4658 4532 7594 7556 3970 7604 6879 3712
 7567 7506 7513 5690 5015 4530 6381 6005 7505   55 2964 7020 7311 2205
 7162 6518 5770 4156 5782 5009 6354 3443 4771 4592 7296 6835 7576 7074
 6340 2419 6864 4549 5243 4586 7874 8050 6616 7589 1625  959 2805 3811
 4398 6222 5615 5620 6508 4282 7179 4610 5344 5537 6001 4487 7244 3514
 5452 5125 8398 6699 7590 4392 8361 4664 8309 6818 4499 4256 1404 4369
 8031 3588 5641 8412  407 6779 6506 4550 1672  110 8038 6712 5025  298
 7427 7888   62 2405  308 6004 6086 2123 2862 6973 1516 4742 3885  265
 6050 3702  848 5436  644 3964 6878 4186 5078 5490 6948 7342 8390  267
 6200 5466 5182 7571 3680   40 4885 6320 4254 7396 3775 4958 1419 8081
 5564 2688 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for f in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  dbert_model = TFBertModel.from_pretrained('bert-base-uncased')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  filenumbers=summarized_data['filenum']
  len(sentences),len(labels),len(filenumbers)
  
  model_5=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  filenumbers=[int(i) for i in filenumbers]
  filenumbers=np.array(filenumbers)
  print(labels)
  print(filenumbers)

  #train_inp,val_inp,train_label,val_label,train_mask,val_mask,train_fnum,val_fnum=train_test_split(input_ids,labels,attention_masks,filenumbers,test_size=0.1,random_state=42)

  new_train_inp_5=[]
  new_val_inp_5=[]
  new_train_label_5=[]
  new_val_label_5=[]
  new_train_mask_5=[]
  new_train_fnum_5=[]
  new_val_fnum_5=[]
  new_val_mask_5=[]
  for i in range(len(train_fnum)):
    if train_fnum[i] in filenumbers:
      new_train_inp_5.append(train_inp[i])
      #new_val_inp.append(val_inp[i])
      new_train_label_5.append(train_label[i])
      #new_val_label.append(val_label[i])
      new_train_mask_5.append(train_mask[i])
      #new_val_mask.append(val_mask[i])
      new_train_fnum_5.append(train_fnum[i])

  for i in range(len(val_fnum)):
    if val_fnum[i] in filenumbers:
      #new_train_inp.append(train_inp[i])
      new_val_inp_5.append(val_inp[i])
      #new_train_label.append(train_label[i])
      new_val_label_5.append(val_label[i])
      #new_train_mask.append(train_mask[i])
      new_val_mask_5.append(val_mask[i])
      new_val_fnum_5.append(val_fnum[i])

  # print(new_train_label)
  # print(len(new_train_label))
  # print(new_val_label)
  # print(len(new_val_label))
  new_train_inp_5=np.array(new_train_inp_5)
  new_val_inp_5=np.array(new_val_inp_5)
  new_train_label_5=np.array(new_train_label_5)
  new_val_label_5=np.array(new_val_label_5)
  new_train_mask_5=np.array(new_train_mask_5)
  new_train_fnum_5=np.array(new_train_fnum_5)
  new_val_fnum_5=np.array(new_val_fnum_5)
  new_val_mask_5=np.array(new_val_mask_5)

  print(new_val_fnum_5)
  
  log_dir='dbert_model'

  model_save_path='./drive/MyDrive/Ensemble/bert-ensemble-512-model_5-'+str(f)+'-279labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_5.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  
  history=model_5.fit([new_train_inp_5,new_train_mask_5],new_train_label_5,batch_size=8,epochs=5,validation_data=([new_val_inp_5,new_val_mask_5],new_val_label_5),callbacks=callbacks)

  pred_labels=[]

  model_saved_5= create_model()
  model_saved_5.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved_5.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_4-'+str(f)+'-279labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved_5.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ...  26   4  57]
[   1    2    4 ... 8414 8417 8406]
[6622   35 6713 4989 8233 7444 5061 8344 5464  317 4515 3001 6991 2875
 3885 6647 4857 4504  214 8340 2969 3847 4609 7358 2752 7651 8188 7073
 6483 4952 5072 1967 4071 7590 5030 7297  626 6029 6302 7256 3993 4263
 3835 4293 8129 4837 3843 1418 5274 6205 5997 6685 3310 7721 2360 7197
 4252 8343  909 4354  649 3488 2496 5765 4004 4052 1403 6697 6293 6380
 4602 6057 4236 7978 5083 6552  361 7706 4186 8351  812 6600 8264 8064
 7121 8279 5556 5119 7947 3350 7152 4282 5025 5051 3615 8062 1489 7351
  234 7416 7921 6574 6257 8396 4520 3300 5640  240 7810 1321 6580 5404
 8273 6742 7861 7975 1018 2140 7061 7767 4990 8124 5927 6051 6916 2846
 6818 7052 7775 7582 8116 4767 5248 6055 8331 1564 7326 3065 6764 6854
 4915 3583 5902 4229 7565 3712 7224 4502 3827 6525 6318 2686 5369 8235
 3351 8021  102  224 8294 7707 2985 5146 3709 6824 2281 6837 5436 2649
 4886  966 5476 8387 6166 4387  350 6123 5236 5041  734 3412 6085  489
 4982 5115 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[209  63 108 ...  26   4  57]
[   1    2    4 ... 8414 8417 8406]
[6622   35 6713 4989 8233 7444 5061 8344 5464  317 4515 3001 6991 2875
 3885 6647 4857 4504  214 8340 2969 3847 4609 7358 2752 7651 8188 7073
 6483 4952 5072 1967 4071 7590 5030 7297  626 6029 6302 7256 3993 4263
 3835 4293 8129 4837 3843 1418 5274 6205 5997 6685 3310 7721 2360 7197
 4252 8343  909 4354  649 3488 2496 5765 4004 4052 1403 6697 6293 6380
 4602 6057 4236 7978 5083 6552  361 7706 4186 8351  812 6600 8264 8064
 7121 8279 5556 5119 7947 3350 7152 4282 5025 5051 3615 8062 1489 7351
  234 7416 7921 6574 6257 8396 4520 3300 5640  240 7810 1321 6580 5404
 8273 6742 7861 7975 1018 2140 7061 7767 4990 8124 5927 6051 6916 2846
 6818 7052 7775 7582 8116 4767 5248 6055 8331 1564 7326 3065 6764 6854
 4915 3583 5902 4229 7565 3712 7224 4502 3827 6525 6318 2686 5369 8235
 3351 8021  102  224 8294 7707 2985 5146 3709 6824 2281 6837 5436 2649
 4886  966 5476 8387 6166 4387  350 6123 5236 5041  734 3412 6085  489
 4982 5115 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


509/509 [==============================] - 193s 346ms/step - loss: 13.5783 - accuracy: 0.1171 - val_loss: 11.6291 - val_accuracy: 0.3075
Epoch 2/5
509/509 [==============================] - 176s 346ms/step - loss: 10.8749 - accuracy: 0.3589 - val_loss: 10.1993 - val_accuracy: 0.3918
Epoch 3/5
509/509 [==============================] - 176s 345ms/step - loss: 9.4330 - accuracy: 0.4836 - val_loss: 9.2554 - val_accuracy: 0.4396
Epoch 4/5
509/509 [==============================] - 176s 345ms/step - loss: 8.3674 - accuracy: 0.5803 - val_loss: 8.6404 - val_accuracy: 0.4784
Epoch 5/5
509/509 [==============================] - 172s 338ms/step - loss: 7.5228 - accuracy: 0.6571 - val_loss: 8.1247 - val_accuracy: 0.4784
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512)]        0           [

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

def load_model(loss,accuracy,optimizer,id):
  model= create_model()
  model.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model.load_weights('./drive/MyDrive/Ensemble/bert-ensemble-512-model_'+str(id)+'-279labels.h5')
  return model


In [20]:

final_pred_0=[]

print(new_val_fnum_0)
print(new_val_fnum_1)
print(new_val_fnum_2)
print(new_val_fnum_3)
print(new_val_fnum_4)
print(new_val_fnum_5)

num_correct=0
model_0_0=load_model(loss,accuracy,optimizer,'0-0')
mod_labels_0=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    pred_test_0_0=model_0_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0_0 = pred_test_0_0.argmax(axis=1)
    mod_labels_0.append(pred_labels_0_0[0])
  else:
    mod_labels_0.append(-1)

model_1_0=load_model(loss,accuracy,optimizer,'1-0')
mod_labels_1=[]
for i in range(len(new_val_fnum_0)):
 
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1_0=model_1_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1_0 = pred_test_1_0.argmax(axis=1)
    mod_labels_1.append(pred_labels_1_0[0])
  else:
    mod_labels_1.append(-1)

model_2_0=load_model(loss,accuracy,optimizer,'2-0')
mod_labels_2=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2_0=model_2_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2_0 = pred_test_2_0.argmax(axis=1)
    mod_labels_2.append(pred_labels_2_0[0])
  else:
    mod_labels_2.append(-1)

model_3_0=load_model(loss,accuracy,optimizer,'3-0')
mod_labels_3=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3_0=model_3_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3_0 = pred_test_3_0.argmax(axis=1)
    mod_labels_3.append(pred_labels_3_0[0])
  else:
    mod_labels_3.append(-1)

model_4_0=load_model(loss,accuracy,optimizer,'4-0')
mod_labels_4=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4_0=model_4_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4_0 = pred_test_4_0.argmax(axis=1)
    mod_labels_4.append(pred_labels_4_0[0])
  else:
    mod_labels_4.append(-1)

model_5_0=load_model(loss,accuracy,optimizer,'5-0')
mod_labels_5=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5_0=model_5_0.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5_0 = pred_test_5_0.argmax(axis=1)
    mod_labels_5.append(pred_labels_5_0[0])
  else:
    mod_labels_5.append(-1)

print(mod_labels_0)
print(mod_labels_1)
print(mod_labels_2)
print(mod_labels_3)
print(mod_labels_4)
print(mod_labels_5)


for i in range(len(new_val_fnum_0)):
  fin_labels=[]
  if mod_labels_0[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_1[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_2[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_3[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_4[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_5[i] != -1:
    fin_labels.append(mod_labels_0[i])
  res=max(set(fin_labels), key = fin_labels.count)
  final_pred_0.append(res)

  if res==new_val_label_0[i]:
    num_correct=num_correct+1

accuracy=num_correct/len(new_val_label_0)
print(accuracy)
  
print(final_pred_0)


[6622   35 2227 6713 4989 8233 7444 5061 3942 8344  454 5464  317 2883
 4515  796 3001 3410 6991 2875 1125 1657  354 3885 6647 4857 4504  214
 3230 8340 2969 3847 4609 5092 6110 1346 5993 7358 2752 7651 7301 8188
 7073 4782 6069 2346 6445 6483  542 1663 4952 5072 1967 4071 7590  860
 6418 1997 5030 7297 2570  626 5463 6029 6302 3819 7256 2950 5667 1235
 3993 1439 1323 4263 3835 2134 3054 2624 5801 4293 8129  320 5866 3247
 5777 2373 5518   19 6059 6520 4837 3843 5695 1666  858 1418 5409 5274
 6205 3115 3167 2356 8418 5997 6685  223 3444 3310 7721 5928 6139 2360
 7197 4252 8343  909 4093 4354 4752 1421  649  829 1916 1003 3488 2496
 5765 6612 4004 4052 1403 6697 5268 1010 6293  264 6380 4602 6057 1480
 1851 4577 4236 7978 1350 5256 5083 6552 1058  361 7706 4186 2590 4452
 4396 5386 2522  749  870 8351 3834  812 5108  463 2274 1743 6600 8264
 8064 7121 8279 5556 1845 5119 2516 4673 2652 7947 8215 1882 3350 7617
 7152 5199 3965 1374 4282 7672 1092 3342 8003 5025 5051  535 6024  933
 3969 

In [21]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

accuracy=accuracy_score(new_val_label_0, final_pred_0)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(new_val_label_0,final_pred_0, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(new_val_label_0,final_pred_0, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(new_val_label_0, final_pred_0, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(new_val_label_0, final_pred_0, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(new_val_label_0, final_pred_0, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(new_val_label_0, final_pred_0, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Accuracy: 0.5320665083135392
Weighted F1: 0.48546204832823997
Micro F1: 0.5320665083135392
Weighted Precision: 0.4800951631478601
Micro Precision: 0.5320665083135392
Weighted Recall: 0.5320665083135392
Micro Recall: 0.5320665083135392


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:


final_pred_1=[]

num_correct=0
model_0_1=load_model(loss,accuracy,optimizer,'0-1')
mod_labels_0=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    pred_test_0_1=model_0_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0_1 = pred_test_0_1.argmax(axis=1)
    mod_labels_0.append(pred_labels_0_1[0])
  else:
    mod_labels_0.append(-1)

model_1_1=load_model(loss,accuracy,optimizer,'1-1')
mod_labels_1=[]
for i in range(len(new_val_fnum_0)):
 
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1_1=model_1_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1_1 = pred_test_1_1.argmax(axis=1)
    mod_labels_1.append(pred_labels_1_1[0])
  else:
    mod_labels_1.append(-1)

model_2_1=load_model(loss,accuracy,optimizer,'2-1')
mod_labels_2=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2_1=model_2_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2_1 = pred_test_2_1.argmax(axis=1)
    mod_labels_2.append(pred_labels_2_1[0])
  else:
    mod_labels_2.append(-1)

model_3_1=load_model(loss,accuracy,optimizer,'3-1')
mod_labels_3=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3_1=model_3_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3_1 = pred_test_3_1.argmax(axis=1)
    mod_labels_3.append(pred_labels_3_1[0])
  else:
    mod_labels_3.append(-1)

model_4_1=load_model(loss,accuracy,optimizer,'4-1')
mod_labels_4=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4_1=model_4_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4_1 = pred_test_4_1.argmax(axis=1)
    mod_labels_4.append(pred_labels_4_1[0])
  else:
    mod_labels_4.append(-1)

model_5_1=load_model(loss,accuracy,optimizer,'5-1')
mod_labels_5=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5_1=model_5_1.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5_1 = pred_test_5_1.argmax(axis=1)
    mod_labels_5.append(pred_labels_5_1[0])
  else:
    mod_labels_5.append(-1)

print(mod_labels_0)
print(mod_labels_1)
print(mod_labels_2)
print(mod_labels_3)
print(mod_labels_4)
print(mod_labels_5)


for i in range(len(new_val_fnum_0)):
  fin_labels=[]
  if mod_labels_0[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_1[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_2[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_3[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_4[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_5[i] != -1:
    fin_labels.append(mod_labels_0[i])
  res=max(set(fin_labels), key = fin_labels.count)
  final_pred_1.append(res)

  if res==new_val_label_0[i]:
    num_correct=num_correct+1

accuracy=num_correct/len(new_val_label_0)
print(accuracy)
  
print(final_pred_1)


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_15[0][0]',               
                                thPoolingAndCrossAt               'input_16[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [23]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


accuracy=accuracy_score(new_val_label_0, final_pred_1)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(new_val_label_0,final_pred_1, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(new_val_label_0,final_pred_1, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(new_val_label_0, final_pred_1, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(new_val_label_0, final_pred_1, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(new_val_label_0, final_pred_1, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(new_val_label_0, final_pred_1, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Accuracy: 0.5118764845605701
Weighted F1: 0.46149808622998945
Micro F1: 0.5118764845605701
Weighted Precision: 0.45922340830262337
Micro Precision: 0.5118764845605701
Weighted Recall: 0.5118764845605701
Micro Recall: 0.5118764845605701


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:


final_pred_2=[]

num_correct=0
model_0_2=load_model(loss,accuracy,optimizer,'0-2')
mod_labels_0=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    pred_test_0_2=model_0_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0_2 = pred_test_0_2.argmax(axis=1)
    mod_labels_0.append(pred_labels_0_2[0])
  else:
    mod_labels_0.append(-1)

model_1_2=load_model(loss,accuracy,optimizer,'1-2')
mod_labels_1=[]
for i in range(len(new_val_fnum_0)):
 
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1_2=model_1_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1_2 = pred_test_1_2.argmax(axis=1)
    mod_labels_1.append(pred_labels_1_2[0])
  else:
    mod_labels_1.append(-1)

model_2_2=load_model(loss,accuracy,optimizer,'2-2')
mod_labels_2=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2_2=model_2_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2_2 = pred_test_2_2.argmax(axis=1)
    mod_labels_2.append(pred_labels_2_2[0])
  else:
    mod_labels_2.append(-1)

model_3_2=load_model(loss,accuracy,optimizer,'3-2')
mod_labels_3=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3_2=model_3_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3_2 = pred_test_3_2.argmax(axis=1)
    mod_labels_3.append(pred_labels_3_2[0])
  else:
    mod_labels_3.append(-1)

model_4_2=load_model(loss,accuracy,optimizer,'4-2')
mod_labels_4=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4_2=model_4_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4_2 = pred_test_4_2.argmax(axis=1)
    mod_labels_4.append(pred_labels_4_2[0])
  else:
    mod_labels_4.append(-1)

model_5_2=load_model(loss,accuracy,optimizer,'5-2')
mod_labels_5=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5_2=model_5_2.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5_2 = pred_test_5_2.argmax(axis=1)
    mod_labels_5.append(pred_labels_5_2[0])
  else:
    mod_labels_5.append(-1)

print(mod_labels_0)
print(mod_labels_1)
print(mod_labels_2)
print(mod_labels_3)
print(mod_labels_4)
print(mod_labels_5)


for i in range(len(new_val_fnum_0)):
  fin_labels=[]
  if mod_labels_0[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_1[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_2[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_3[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_4[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_5[i] != -1:
    fin_labels.append(mod_labels_0[i])
  res=max(set(fin_labels), key = fin_labels.count)
  final_pred_2.append(res)

  if res==new_val_label_0[i]:
    num_correct=num_correct+1

accuracy=num_correct/len(new_val_label_0)
print(accuracy)
  
print(final_pred_2)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_28 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_27[0][0]',               
                                thPoolingAndCrossAt               'input_28[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                        

In [25]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


accuracy=accuracy_score(new_val_label_0, final_pred_2)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(new_val_label_0,final_pred_2, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(new_val_label_0,final_pred_2, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(new_val_label_0, final_pred_2, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(new_val_label_0, final_pred_2, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(new_val_label_0, final_pred_2, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(new_val_label_0, final_pred_2, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Accuracy: 0.5320665083135392
Weighted F1: 0.4839795590444909
Micro F1: 0.5320665083135392
Weighted Precision: 0.4809378256716375
Micro Precision: 0.5320665083135392
Weighted Recall: 0.5320665083135392
Micro Recall: 0.5320665083135392


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:


final_pred_3=[]

num_correct=0
model_0_3=load_model(loss,accuracy,optimizer,'0-3')
mod_labels_0=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    pred_test_0_3=model_0_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0_3 = pred_test_0_3.argmax(axis=1)
    mod_labels_0.append(pred_labels_0_3[0])
  else:
    mod_labels_0.append(-1)

model_1_3=load_model(loss,accuracy,optimizer,'1-3')
mod_labels_1=[]
for i in range(len(new_val_fnum_0)):
 
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1_3=model_1_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1_3 = pred_test_1_3.argmax(axis=1)
    mod_labels_1.append(pred_labels_1_3[0])
  else:
    mod_labels_1.append(-1)

model_2_3=load_model(loss,accuracy,optimizer,'2-3')
mod_labels_2=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2_3=model_2_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2_3 = pred_test_2_3.argmax(axis=1)
    mod_labels_2.append(pred_labels_2_3[0])
  else:
    mod_labels_2.append(-1)

model_3_3=load_model(loss,accuracy,optimizer,'3-3')
mod_labels_3=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3_3=model_3_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3_3 = pred_test_3_3.argmax(axis=1)
    mod_labels_3.append(pred_labels_3_3[0])
  else:
    mod_labels_3.append(-1)

model_4_3=load_model(loss,accuracy,optimizer,'4-3')
mod_labels_4=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4_3=model_4_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4_3 = pred_test_4_3.argmax(axis=1)
    mod_labels_4.append(pred_labels_4_3[0])
  else:
    mod_labels_4.append(-1)

model_5_3=load_model(loss,accuracy,optimizer,'5-3')
mod_labels_5=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5_3=model_5_3.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5_3 = pred_test_5_3.argmax(axis=1)
    mod_labels_5.append(pred_labels_5_3[0])
  else:
    mod_labels_5.append(-1)

print(mod_labels_0)
print(mod_labels_1)
print(mod_labels_2)
print(mod_labels_3)
print(mod_labels_4)
print(mod_labels_5)


for i in range(len(new_val_fnum_0)):
  fin_labels=[]
  if mod_labels_0[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_1[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_2[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_3[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_4[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_5[i] != -1:
    fin_labels.append(mod_labels_0[i])
  res=max(set(fin_labels), key = fin_labels.count)
  final_pred_3.append(res)

  if res==new_val_label_0[i]:
    num_correct=num_correct+1

accuracy=num_correct/len(new_val_label_0)
print(accuracy)
  
print(final_pred_3)

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_39 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_40 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_39[0][0]',               
                                thPoolingAndCrossAt               'input_40[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                        

In [27]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


accuracy=accuracy_score(new_val_label_0, final_pred_3)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(new_val_label_0,final_pred_3, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(new_val_label_0,final_pred_3, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(new_val_label_0, final_pred_3, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(new_val_label_0, final_pred_3, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(new_val_label_0, final_pred_3, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(new_val_label_0, final_pred_3, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Accuracy: 0.517814726840855
Weighted F1: 0.46327971731288403
Micro F1: 0.517814726840855
Weighted Precision: 0.44880169390373914
Micro Precision: 0.517814726840855
Weighted Recall: 0.517814726840855
Micro Recall: 0.517814726840855


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:


final_pred_4=[]

num_correct=0
model_0_4=load_model(loss,accuracy,optimizer,'0-4')
mod_labels_0=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_0:
    pred_test_0_4=model_0_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_0_4 = pred_test_0_4.argmax(axis=1)
    mod_labels_0.append(pred_labels_0_4[0])
  else:
    mod_labels_0.append(-1)

model_1_4=load_model(loss,accuracy,optimizer,'1-4')
mod_labels_1=[]
for i in range(len(new_val_fnum_0)):
 
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_1:
    pred_test_1_4=model_1_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_1_4 = pred_test_1_4.argmax(axis=1)
    mod_labels_1.append(pred_labels_1_4[0])
  else:
    mod_labels_1.append(-1)

model_2_4=load_model(loss,accuracy,optimizer,'2-4')
mod_labels_2=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_2:
    pred_test_2_4=model_2_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_2_4 = pred_test_2_4.argmax(axis=1)
    mod_labels_2.append(pred_labels_2_4[0])
  else:
    mod_labels_2.append(-1)

model_3_4=load_model(loss,accuracy,optimizer,'3-4')
mod_labels_3=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_3:
    pred_test_3_4=model_3_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_3_4 = pred_test_3_4.argmax(axis=1)
    mod_labels_3.append(pred_labels_3_4[0])
  else:
    mod_labels_3.append(-1)

model_4_4=load_model(loss,accuracy,optimizer,'4-4')
mod_labels_4=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_4:
    pred_test_4_4=model_4_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_4_4 = pred_test_4_4.argmax(axis=1)
    mod_labels_4.append(pred_labels_4_4[0])
  else:
    mod_labels_4.append(-1)

model_5_4=load_model(loss,accuracy,optimizer,'5-4')
mod_labels_5=[]
for i in range(len(new_val_fnum_0)):
  
  #print(i)
  if new_val_fnum_0[i] in new_val_fnum_5:
    pred_test_5_4=model_5_4.predict([new_val_inp_0[i].reshape(1,512),new_val_mask_0[i].reshape(1,512)])
    pred_labels_5_4 = pred_test_5_4.argmax(axis=1)
    mod_labels_5.append(pred_labels_5_4[0])
  else:
    mod_labels_5.append(-1)

print(mod_labels_0)
print(mod_labels_1)
print(mod_labels_2)
print(mod_labels_3)
print(mod_labels_4)
print(mod_labels_5)


for i in range(len(new_val_fnum_0)):
  fin_labels=[]
  if mod_labels_0[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_1[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_2[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_3[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_4[i] != -1:
    fin_labels.append(mod_labels_0[i])
  if mod_labels_5[i] != -1:
    fin_labels.append(mod_labels_0[i])
  res=max(set(fin_labels), key = fin_labels.count)
  final_pred_4.append(res)

  if res==new_val_label_0[i]:
    num_correct=num_correct+1

accuracy=num_correct/len(new_val_label_0)
print(accuracy)
  
print(final_pred_4)

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_51 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 input_52 (InputLayer)          [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_51[0][0]',               
                                thPoolingAndCrossAt               'input_52[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                        

In [29]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


accuracy=accuracy_score(new_val_label_0, final_pred_4)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(new_val_label_0,final_pred_4, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(new_val_label_0,final_pred_4, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(new_val_label_0, final_pred_4, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(new_val_label_0, final_pred_4, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(new_val_label_0, final_pred_4, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(new_val_label_0, final_pred_4, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Accuracy: 0.5083135391923991
Weighted F1: 0.4617582441807906
Micro F1: 0.5083135391923991
Weighted Precision: 0.4533129233242081
Micro Precision: 0.5083135391923991
Weighted Recall: 0.5083135391923991
Micro Recall: 0.5083135391923991


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))

Average Accuracy: 0.5204275534441805
Average Weighted F1: 0.471195531019279
Average Micro F1: 0.5204275534441805
Average Weighted Precision: 0.4644742028700136
Average Micro Precision: 0.5204275534441805
Average Weighted Recall: 0.5204275534441805
Average Micro Recall: 0.5204275534441805
